In [1]:
import os
import pandas as pd
import re

input_folder = "/home/hoai-linh.dao/Works/EVCS/EV-Simulations/data/processed-data/total_scenario/old-form-all"
csv_output_folder = "/home/hoai-linh.dao/Works/EVCS/EV-Simulations/data/processed-data/total_scenario/region-based/csv"
split_output_folder = "/home/hoai-linh.dao/Works/EVCS/EV-Simulations/data/processed-data/total_scenario/region-based/ods"

os.makedirs(csv_output_folder, exist_ok=True)
os.makedirs(split_output_folder, exist_ok=True)

zones = list(range(1, 7)) + [99]
zone_data = {zone: {"hour": [], "intra": [], "inter": [], "detail": []} for zone in zones}

for z in zones:
    os.makedirs(os.path.join(split_output_folder, f"zone_{z}"), exist_ok=True)

for hour in range(24):
    file_name = f"od_matrix_{hour:02}.txt"
    input_path = os.path.join(input_folder, file_name)

    if not os.path.exists(input_path):
        continue

    with open(input_path, "r") as f:
        lines = f.readlines()

    od_entries = []
    for line in lines:
        parts = re.findall(r"\d+", line)
        if len(parts) == 3:
            origin, dest, trips = map(int, parts)
            od_entries.append((origin, dest, trips))

    for zone in zones:
        intra = 0
        inter = 0
        outbound = 0
        inbound = 0
        detail_dict = {f"{zone}<->{other}": 0 for other in zones if other != zone}

        for origin, dest, trips in od_entries:
            if origin == zone and dest == zone:
                intra += trips
            elif origin == zone and dest in zones:
                inter += trips
                outbound += trips
                detail_dict[f"{zone}<->{dest}"] += trips
            elif dest == zone and origin in zones:
                inter += trips
                inbound += trips
                detail_dict[f"{zone}<->{origin}"] += trips

        zone_data[zone]["hour"].append(hour)
        zone_data[zone]["intra"].append(intra)
        zone_data[zone]["inter"].append(inter)
        zone_data[zone]["detail"].append(detail_dict)

        zz = zone * 10 + zone
        output_path = os.path.join(split_output_folder, f"zone_{zone}", file_name)
        with open(output_path, "w") as out:
            out.write("$OR;D2\n")
            out.write(f"{hour:02d}.00 {hour+1:02d}.00\n")
            out.write(f"{zone:.2f}\n")
            out.write(f"{zone:10d}{zone:10d}{intra:10d}\n")
            out.write(f"{zone:10d}{zz:10d}{outbound:10d}\n")
            out.write(f"{zz:10d}{zone:10d}{inbound:10d}\n")

for zone in zones:
    df = pd.DataFrame(zone_data[zone])
    detail_df = pd.DataFrame(df["detail"].tolist())
    final_df = pd.concat([df.drop(columns=["detail"]), detail_df], axis=1)

    csv_path = os.path.join(csv_output_folder, f"{zone}.csv")
    final_df.to_csv(csv_path, index=False)
